In [ ]:
import os
import cv2
from pathlib import Path
import open3d as o3d
import numpy as np
import rootutils
import matplotlib.pyplot as plt

from helpers.segmentation_inference import SegmentationInferenceOnnx, segmentation_inference

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.stereo_inference import StereoInferenceOnnx

from src.stereo_calibrate import (
    read_stereo_calibration,
    rectify_image,
    read_camera_intrinsics,
)
from src.utils import (
    StereoImages,
    visualize_disparity,
    postprocess_disparity,
    create_point_cloud,
    disparity_to_depth_map,
)

#### Loading models

In [ ]:
# Load stereo calibration parameters
stereo_map_left, stereo_map_right = read_stereo_calibration(
    '../camera_configs/stereo_calibration.xml'
)
# Read camera intrinsics
K, distance_between_cameras = read_camera_intrinsics(
    '../camera_configs/left_camera_intrinsics.xml'
)
# Load stereo model
stereo_model = StereoInferenceOnnx('../models/fs_800_640.onnx')

In [ ]:
# mask_path = '../data/left_anomaly_2_mask/left_' + '20250917_153919_0' + '.png'
# mask_path = None
mask_path = 'model'

In [ ]:
if mask_path == 'model':
    # Load segmentation model
    segmentation_model = SegmentationInferenceOnnx(
        model_path='../models/unet++_lab_seat_segmentation.onnx'
    )

#### Loading images

In [ ]:
# load images
image_name = '20250917_154307_2'
left_image_path = '../data/left_anomaly_2/left_' + image_name + '.png'
right_image_path = '../data/right_anomaly_2/right_' + image_name + '.png'

processor = StereoImages(stereo_map_left, stereo_map_right)
proc_result = processor.process(
    left_image_path,
    right_image_path,
    stereo_model.input_width,
    stereo_model.input_height
    )

left_tensor, right_tensor = proc_result['tensors']
scale = proc_result['scale']
mask_rectified = None

# show rectified images
plt.subplot(1, 2, 1)
plt.imshow(proc_result['processed_images'][0])
plt.title('Left Rectified Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(proc_result['processed_images'][1])
plt.title('Right Rectified Image')
plt.axis('off')
plt.show()

#### Segmentation

In [ ]:
if mask_path == 'model':
    left_image = proc_result['raw_images'][0]
    mask = segmentation_inference(segmentation_model, left_image)
    mask_rectified = rectify_image(mask, stereo_map_left)
    mask_rectified = cv2.resize(
        mask_rectified,
        (stereo_model.input_width, stereo_model.input_height),
        interpolation=cv2.INTER_LINEAR,
    )

    # show mask
    plt.imshow(mask_rectified, cmap='gray')
    plt.title('Left Mask Image from file')

elif mask_path is not None:
    # read mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # make non black area 255
    mask[mask > 0] = 255
    # resize mask to model input size
    mask_rectified = rectify_image(mask, stereo_map_left)
    mask_rectified = cv2.resize(
        mask_rectified,
        (stereo_model.input_width, stereo_model.input_height),
        interpolation=cv2.INTER_LINEAR,
    )

    # show mask
    plt.imshow(mask_rectified, cmap='gray')
    plt.title('Left Mask Image from file')

#### Inference

In [ ]:
# Stereo inference
disparity_map = stereo_model(left_tensor, right_tensor)

#### Post processing

In [ ]:
# squeeze form 4D to 2D
disparity_map = disparity_map.squeeze()
# Visualize disparity map
disparity_map_viz = visualize_disparity(disparity_map, mask=mask_rectified)
# Postprocess disparity map
disparity_map_proc = postprocess_disparity(disparity_map, mask=mask_rectified)
# depth map
depth_map = disparity_to_depth_map(disparity_map, K, distance_between_cameras, scale)
# post process depth map
depth_map_proc = postprocess_disparity(depth_map, mask=mask_rectified)

In [ ]:
# show disparity map with matplotlib
plt.figure(figsize=(10, 8))
im1 = plt.imshow(depth_map_proc, cmap='jet')
plt.colorbar(im1, label='Depth (m)')
plt.show()

#### Saving

In [ ]:
# save disparity map as numpy file
np.save(f'../output/disp/disparity_map_{image_name}.npy', depth_map_proc)

In [ ]:
# save disparity map
cv2.imwrite(f'../output/disp/disparity_map_{image_name}.png', cv2.cvtColor(disparity_map_viz, cv2.COLOR_RGB2BGR))

In [ ]:
left_rectified = proc_result['rectified_images'][0]

# Create point cloud
pcd = create_point_cloud(
    disparity_map_proc, left_rectified, K, distance_between_cameras, scale=scale
)

# Save point cloud to PLY file
output_path = Path('../output/pcd/')
output_path.mkdir(parents=True, exist_ok=True)
o3d.io.write_point_cloud(str(output_path / (image_name + '.ply')), pcd)